## Country-level INFRA-SAP

Create summary statistics at admin-2 and country-level

In [2]:
import os, sys, time, importlib

import geopandas as gpd
import pandas as pd
import networkx as nx
sys.path.append('/home/wb514197/Repos/GOSTnets')

import GOSTnets as gn
import rasterio as rio
from osgeo import gdal
import GOSTnets.calculate_od_raw as calcOD
import numpy as np

%load_ext autoreload
%autoreload 2

sys.path.append('/home/wb514197/Repos/INFRA_SAP')
from infrasap import aggregator
from shapely.wkt import loads
import infrasap.rai_calculator as rai

In [3]:
country = 'zimbabwe'
iso3 = 'ZWE'
# epsg = 32638

### Load data

In [4]:
# base_in = "/home/public/Data/PROJECTS/INFRA_SAP"
# in_folder = os.path.join(base_in, iso3)

# # define data paths
# focal_admin2 = os.path.join(in_folder, "admin.shp")
# focal_osm = os.path.join(in_folder, f"{country}-latest.osm.pbf")
# pop_name = "WP_2020_1km"
# wp_1km = os.path.join(in_folder, f"{pop_name}.tif")
# urban_extents = os.path.join(in_folder, "urban_extents.shp")
# airports = os.path.join(in_folder, "airports_intl.shp")
# ports = os.path.join(in_folder, "ports.shp")
# borders = os.path.join(in_folder, "borders.shp")

In [5]:
# out_folder = "/home/wb514197/data/uruguay"
# if not os.path.exists(out_folder):
#     os.makedirs(out_folder)

In [63]:
# in_folder = "/home/public/Data/PROJECTS/infraSAP_URY/origins"
# pop_name = "ury_ppp_2020_UNadj_agg6"
# wp_1km = os.path.join(in_folder, f"{pop_name}.tif")

In [5]:
base_in = "/home/public/Data/PROJECTS/INFRA_SAP"
# base_in = "/home/wb514197/data/INFRA_SAP"
in_folder = os.path.join(base_in, iso3)

# define data paths
focal_admin2 = os.path.join(in_folder, "admin.shp")
focal_osm = os.path.join(in_folder, f"{country}-latest.osm.pbf")
pop_name = "WP_2020_1km"
wp_1km = os.path.join(in_folder, f"{pop_name}.tif")
urban_extents = os.path.join(in_folder, "urban_extents.shp")
airports = os.path.join(in_folder, "airports_intl.shp")
ports = os.path.join(in_folder, "ports.shp")
borders = os.path.join(in_folder, "borders.shp")

base_out = "/home/wb514197/data/INFRA_SAP" # GOT permission denied using public 
out_folder = os.path.join(base_out, iso3)
# if not os.path.exists(out_folder):
#     os.makedirs(out_folder)

# out_folder = os.path.join(in_folder, "output")
# out_folder = os.path.join(in_folder, "out")

In [6]:
tables_path = os.path.join(out_folder, 'tables', 'new')
if not os.path.exists(tables_path):
    os.mkdir(tables_path)

In [7]:
target = gpd.read_file(focal_admin2)

### 1. RAI

In [7]:
%%time
rai_roadnetwork = rai.extract_rai_network(focal_osm, epsg=epsg)

/home/wb514197/.conda/envs/geo/lib/python3.7/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/wb514197/.conda/envs/geo/lib/python3.7/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


CPU times: user 41.6 s, sys: 392 ms, total: 42 s
Wall time: 41.8 s


In [8]:
rai_roadnetwork = rai_roadnetwork.to_crs('EPSG:4326')

In [9]:
%%time
rai = rai.calculate_rai(target, "OBJECTID", rio.open(wp_1km), rai_roadnetwork, tables_path)

CPU times: user 15.8 s, sys: 23.9 ms, total: 15.9 s
Wall time: 15.9 s


In [10]:
rai.head()

,RAI_POP_1,ID,RAI_POP_2,RAI_POP_3,RAI_POP_4,POP
0,21231.535156,14067,79563.390625,101522.109375,113246.218750,139532.656250
1,2110.793457,14068,13114.705078,13696.375977,21390.917969,39845.968750
2,27424.113281,14069,27555.964844,28926.244141,32268.984375,48604.304688
3,11647.585938,14070,14191.382812,17667.683594,20671.757812,52190.277344
4,633021.625000,14071,649223.750000,669919.937500,674469.375000,677517.062500


In [18]:
rai.loc[rai.RAI_POP_1<0, "RAI_POP_1"] = 0

In [21]:
rai['RAI_Percent_1'] = rai.RAI_POP_1/rai.POP
rai['RAI_Percent_2'] = rai.RAI_POP_2/rai.POP
rai['RAI_Percent_3'] = rai.RAI_POP_3/rai.POP
rai['RAI_Percent_4'] = rai.RAI_POP_4/rai.POP

In [22]:
# don't need this anymore
rai.rename(columns={'ID':'OBJECTID'}, inplace=True)
rai = rai[['OBJECTID','POP','RAI_POP_1', 'RAI_POP_2', 'RAI_POP_3', 'RAI_POP_4',
           'RAI_Percent_1', 'RAI_Percent_2', 'RAI_Percent_3', 'RAI_Percent_4']]
rai = target.merge(rai, on="OBJECTID")
rai.drop(['geometry','Shape_Leng','Shape_Area'], axis=1, inplace=True)

In [23]:
rai.to_excel(os.path.join(tables_path, "1Rural Access to Roads.xlsx"), index=False)

### 2. Reload OD Matrix

In [30]:
OD = pd.read_csv(os.path.join(out_folder, 'OD_09_09.csv'), header=[0,1], index_col=0)

In [32]:
OD['city'] = OD['city'].apply(lambda x: (x/3600))
# OD['port'] = OD['port'].apply(lambda x: (x/3600))
OD['border'] = OD['border'].apply(lambda x: (x/3600))
OD['airport'] = OD['airport'].apply(lambda x: (x/3600))

### 3. District-level summaries

**Percentage of rural population with access to a main city (>50k inhabitants) in XX hours**

In [34]:
dest_type='city'
rural=True
thresholds=[0,0.5,1,1.5,2,3,4,5,6,7,10,1000] # hours
rural_access = aggregator.calculate_access_percentages(OD, target, 'city', True, urban_extents, thresholds=thresholds)

In [35]:
rural_access.rename(columns = {
    '(10.0, 1000.0]':'> 10.0',
    '(10.0, 1000.0] pop':'> 10.0 pop'
}, inplace=True)
rural_access.drop(columns=['geometry','Shape_Leng','Shape_Area'], inplace=True)

In [36]:
tables_path

'/home/wb514197/data/INFRA_SAP/ZWE/tables/new'

In [37]:
rural_access.to_excel(os.path.join(tables_path, "2Rural Access to Cities.xlsx"), index=False)

**Travel time to closest city**

In [38]:
city_tt = aggregator.calculate_access_percentages(OD, target, 'city', thresholds=thresholds)
city_tt.rename(columns = {
    '(10.0, 1000.0]':'> 10.0',
    '(10.0, 1000.0] pop':'> 10.0 pop'
}, inplace=True)
city_tt.drop(columns=['geometry','Shape_Leng','Shape_Area'], inplace=True)
city_tt.to_excel(os.path.join(tables_path, "3Travel Time to Nearest City.xlsx"), index=False)

**Travel time to capital**

In [39]:
dest_all = pd.read_csv(os.path.join(out_folder, 'destination_all.csv'), index_col=0)
cap_idx = dest_all.sort_values('Pop', ascending=False).iloc[[0]].index[0]

In [40]:
capital_tt = aggregator.calculate_access_percentages(OD, target, 'city', thresholds=thresholds, capital=str(cap_idx))
capital_tt.rename(columns = {
    '(10.0, 1000.0]':'> 10.0',
    '(10.0, 1000.0] pop':'> 10.0 pop'
}, inplace=True)
capital_tt.drop(columns=['geometry','Shape_Leng','Shape_Area'], inplace=True)
capital_tt.to_excel(os.path.join(tables_path, "4Travel Time to Capital.xlsx"), index=False)

**Time to drive to the closest land border point**

In [41]:
border_tt = aggregator.calculate_access_percentages(OD, target, 'border', thresholds=thresholds)
border_tt.rename(columns = {
    '(10.0, 1000.0]':'> 10.0',
    '(10.0, 1000.0] pop':'> 10.0 pop'
}, inplace=True)
border_tt.drop(columns=['geometry','Shape_Leng','Shape_Area'], inplace=True)
border_tt.to_excel(os.path.join(tables_path, "6Travel Time to Nearest Border.xlsx"), index=False)

**Time to drive to the closest airport**

In [42]:
airport_tt = aggregator.calculate_access_percentages(OD, target, 'airport', thresholds=thresholds)
airport_tt.rename(columns = {
    '(10.0, 1000.0]':'> 10.0',
    '(10.0, 1000.0] pop':'> 10.0 pop'
}, inplace=True)
airport_tt.drop(columns=['geometry','Shape_Leng','Shape_Area'], inplace=True)
airport_tt.to_excel(os.path.join(tables_path, "5Travel Time to Nearest Airport.xlsx"), index=False)

**Time to drive to the closest port**

In [43]:
port_tt = aggregator.calculate_access_percentages(OD, target, 'port', thresholds=thresholds)
port_tt.rename(columns = {
    '(10.0, 1000.0]':'> 10.0',
    '(10.0, 1000.0] pop':'> 10.0 pop'
}, inplace=True)
port_tt.drop(columns=['geometry','Shape_Leng','Shape_Area'], inplace=True)
port_tt.to_excel(os.path.join(tables_path, "7Travel Time to Nearest Port.xlsx"), index=False)

#### Aggregate to the country-level

In [43]:
country_level = aggregator.agregate_to_country(rural_access, "Rural pop. with access to a main city")
country_level = aggregator.agregate_to_country(city_tt, "Travel time to closest city", country_level)
country_level = aggregator.agregate_to_country(capital_tt, "Travel time to capital", country_level)
country_level = aggregator.agregate_to_country(border_tt, "Travel time to closest border", country_level)
country_level = aggregator.agregate_to_country(airport_tt, "Travel time to closest airport", country_level)
# country_level = aggregator.agregate_to_country(port_tt, "Travel time to closest port", country_level)

In [44]:
country_level

,"(0.0, 0.5] pop","(0.5, 1.0] pop","(1.0, 1.5] pop","(1.5, 2.0] pop","(2.0, 3.0] pop","(3.0, 4.0] pop","(4.0, 5.0] pop","(5.0, 6.0] pop","(6.0, 7.0] pop","(7.0, 10.0] pop",> 10.0 pop
Rural pop. with access to a main city,0.043582,0.090376,0.092618,0.089644,0.187562,0.166555,0.130042,0.085143,0.050613,0.052416,0.011448
Travel time to closest city,0.270737,0.110511,0.064145,0.060745,0.138902,0.117844,0.093140,0.058528,0.036219,0.039083,0.010145
Travel time to capital,0.127394,0.061400,0.017893,0.021021,0.060205,0.070143,0.058886,0.074577,0.081878,0.184630,0.241972
Travel time to closest border,0.022080,0.009728,0.010751,0.012618,0.072935,0.061158,0.077292,0.084638,0.247886,0.306293,0.094621
Travel time to closest airport,0.091966,0.134492,0.028078,0.021049,0.072688,0.083249,0.091962,0.106467,0.101457,0.193902,0.074689


#### Pop-weighted average country-level

In [45]:
raster_path = wp_1km

In [46]:
raster_path

'/home/public/Data/PROJECTS/INFRA_SAP/ZWE/WP_2020_1km.tif'

In [47]:
out_folder

'/home/wb514197/data/INFRA_SAP/ZWE'

In [49]:
national_average = aggregator.pop_weighted_average_national(os.path.join(out_folder,'travel_time','cities_min_tt.tif'), raster_path, 'Travel time to closest city')
national_average = aggregator.pop_weighted_average_national(os.path.join(out_folder,'travel_time','capital_tt.tif'), raster_path, 'Travel time to capital', national_average)
national_average = aggregator.pop_weighted_average_national(os.path.join(out_folder,'travel_time','borders_min_tt.tif'), raster_path, 'Travel time to closest border', national_average)
national_average = aggregator.pop_weighted_average_national(os.path.join(out_folder,'travel_time','airport_min_tt.tif'), raster_path, 'Travel time to closest airport', national_average)
# national_average = aggregator.pop_weighted_average_national(os.path.join(out_folder,'travel_time_k5','port_min_tt.tif'), raster_path, 'Travel time to closest port', national_average)

In [50]:
country_level = country_level.join(national_average)

In [51]:
country_level.rename(columns = {
    '(0.0, 0.5] pop':'(0.0, 0.5]',
    '(0.5, 1.0] pop':'(0.5, 1.0]',
    '(1.0, 1.5] pop':'(1.0, 1.5]',
    '(1.5, 2.0] pop':'(1.5, 2.0]',
    '(2.0, 3.0] pop':'(2.0, 3.0]',
    '(3.0, 4.0] pop':'(3.0, 4.0]',
    '(4.0, 5.0] pop':'(4.0, 5.0]',
    '(5.0, 6.0] pop':'(5.0, 6.0]',
    '(6.0, 7.0] pop':'(6.0, 7.0]',
    '(7.0, 10.0] pop':'(7.0, 10.0]',
    '> 10.0 pop':'> 10.0'  
}, inplace=True)
country_level.to_excel(os.path.join(tables_path, "9National Level Aggregates.xlsx"), index=True)

#### Pop-weighted average by district

In [41]:
weighted_average = aggregator.pop_weighted_average(target, os.path.join(out_folder,'travel_time','cities_min_tt.tif'), raster_path, 'tt_min_city')
weighted_average = aggregator.pop_weighted_average(target, os.path.join(out_folder,'travel_time','capital_tt.tif'), raster_path, 'tt_capital', table=weighted_average)
weighted_average = aggregator.pop_weighted_average(target, os.path.join(out_folder,'travel_time','borders_min_tt.tif'), raster_path, 'tt_border', table=weighted_average)
weighted_average = aggregator.pop_weighted_average(target, os.path.join(out_folder,'travel_time','airport_min_tt.tif'), raster_path, 'tt_airport', table=weighted_average)
weighted_average = aggregator.pop_weighted_average(target, os.path.join(out_folder,'travel_time','port_min_tt.tif'), raster_path, 'tt_port', table=weighted_average)
weighted_average = aggregator.pop_weighted_average(target, os.path.join(out_folder,'travel_distance','cities_min_dist.tif'), raster_path, 'dist_cities', weighted_average)
weighted_average = aggregator.pop_weighted_average(target, os.path.join(out_folder,'travel_distance','airport_min_dist.tif'), raster_path, 'dist_airport', weighted_average)
weighted_average = aggregator.pop_weighted_average(target, os.path.join(out_folder,'travel_distance','port_min_dist.tif'), raster_path, 'dist_ports', weighted_average)
weighted_average = aggregator.pop_weighted_average(target, os.path.join(out_folder,'travel_distance','borders_min_dist.tif'), raster_path, 'dist_borders', weighted_average)
weighted_average = aggregator.pop_weighted_average(target, os.path.join(out_folder,'travel_distance','capital_dist.tif'), raster_path, 'dist_capital',weighted_average)

#### Market Access

In [42]:
out_access = os.path.join(out_folder, 'access')

In [43]:
ma5 = aggregator.pop_weighted_average(target, os.path.join(out_access,f"access_cities_d5.tif"), raster_path, 'ma_5')
ma6 = aggregator.pop_weighted_average(target, os.path.join(out_access,f"access_cities_d6.tif"), raster_path, 'ma_6')
ma7 = aggregator.pop_weighted_average(target, os.path.join(out_access,f"access_cities_d7.tif"), raster_path, 'ma_7')
ma8 = aggregator.pop_weighted_average(target, os.path.join(out_access,f"access_cities_d8.tif"), raster_path, 'ma_8')
ma9 = aggregator.pop_weighted_average(target, os.path.join(out_access,f"access_cities_d9.tif"), raster_path, 'ma_9')

In [44]:
ma = pd.concat([ma5, ma6[['ma_6']], ma7[['ma_7']], ma8[['ma_8']], ma9[['ma_9']]], axis=1)

#### Combine district-level data

In [45]:
district_summary = weighted_average.merge(ma[['OBJECTID','ma_5','ma_6','ma_7','ma_8','ma_9']], on="OBJECTID")

In [46]:
rai.columns

Index(['OBJECTID', 'ISO_A2', 'WB_ADM1_CO', 'WB_ADM0_CO', 'WB_ADM0_NA',
       'WB_ADM1_NA', 'WB_ADM2_CO', 'WB_ADM2_NA', 'ISO3', 'POP', 'RAI_POP_1',
       'RAI_POP_2', 'RAI_POP_3', 'RAI_POP_4', 'RAI_Percent_1', 'RAI_Percent_2',
       'RAI_Percent_3', 'RAI_Percent_4'],
      dtype='object')

In [47]:
# rai = pd.read_excel(os.path.join(tables_path, '1Rural Access to Roads.xlsx'))
rai = rai[['OBJECTID','RAI_Percent_1','RAI_Percent_2', 'RAI_Percent_3', 'RAI_Percent_4']]
district_summary = district_summary.merge(rai, on='OBJECTID')

In [51]:
hubs = pd.read_excel(os.path.join(tables_path, '8Travel Time to Regional Hub.xlsx'))
hubs = hubs.rename(columns={'Time (hrs)':'tt_regional', 'Name1':'hub_name'}) # Name1 'Regional Hub Name'
hubs = hubs[['OBJECTID','tt_regional','hub_name']]
district_summary = district_summary.merge(hubs, on='OBJECTID')

In [52]:
district_summary.to_file(os.path.join(out_folder,'admin_join.shp'), driver='ESRI Shapefile')

In [53]:
district_table = district_summary.copy()
district_table.drop(columns=['geometry','Shape_Leng','Shape_Area'], inplace=True)
district_table.to_excel(os.path.join(tables_path, "10District Level Summary.xlsx"), index=False)

In [54]:
district_summary.columns

Index(['OBJECTID', 'ISO_A2', 'WB_ADM1_CO', 'WB_ADM0_CO', 'WB_ADM0_NA',
       'WB_ADM1_NA', 'WB_ADM2_CO', 'WB_ADM2_NA', 'Shape_Leng', 'Shape_Area',
       'ISO3', 'geometry', 'tt_min_city', 'tt_capital', 'tt_border',
       'tt_airport', 'tt_port', 'dist_cities', 'dist_airport', 'dist_ports',
       'dist_borders', 'dist_capital', 'ma_5', 'ma_6', 'ma_7', 'ma_8', 'ma_9',
       'RAI_Percent_1', 'RAI_Percent_2', 'RAI_Percent_3', 'RAI_Percent_4',
       'tt_regional', 'hub_name'],
      dtype='object')